In [1]:

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [2]:
torch.cuda.is_available()

True

In [5]:
# Create Torch Dataset
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [6]:
train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [7]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2)

C:\Users\tarun\anaconda3\envs\test_env_gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\tarun\anaconda3\envs\test_env_gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [10]:

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [12]:
epochs=5
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        
        print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14038.94921875
Epoch 0, iter 1, loss: 13465.2900390625
Epoch 0, iter 2, loss: 13584.1240234375
Epoch 0, iter 3, loss: 12794.5107421875
Epoch 0, iter 4, loss: 13442.0322265625
Epoch 0, iter 5, loss: 13053.990234375
Epoch 0, iter 6, loss: 13077.810546875
Epoch 0, iter 7, loss: 13830.603515625
Epoch 0, iter 8, loss: 13766.830078125
Epoch 0, iter 9, loss: 13450.994140625
Epoch 0, iter 10, loss: 13721.5830078125
Epoch 0, iter 11, loss: 13763.712890625
Epoch 0, iter 12, loss: 12976.7998046875
Epoch 0, iter 13, loss: 13418.025390625
Epoch 0, iter 14, loss: 12800.1630859375
Epoch 0, iter 15, loss: 13424.48046875
Epoch 0, iter 16, loss: 13155.6220703125
Epoch 0, iter 17, loss: 13061.2138671875
Epoch 0, iter 18, loss: 12634.2822265625
Epoch 0, iter 19, loss: 12959.7373046875
Epoch 0, iter 20, loss: 12397.29296875
Epoch 0, iter 21, loss: 13169.806640625
Epoch 0, iter 22, loss: 13170.021484375
Epoch 0, iter 23, loss: 12707.9482421875
Epoch 0, iter 24, loss: 13213.5224609375


In [14]:
torch.save(model.state_dict(), "keypoints_model.pth")